Two galaxy blends are created and given to WLDEblending package. The Output can then be analyzed with DM stack to study how blending effects depend on the individual galaxy parameters 

## To Do!!
#### Check variance array
#### Add hsm


In [ ]:
import numpy as np
from astropy.table import Table, Column
from scipy import spatial
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import fitsio
import sys
sys.path.insert(0, 'WeakLensingDeblending/')
import descwl
import os
import scipy.ndimage
import galsim
import math

plt.rc('image', origin='lower',
       interpolation='none')
plt.rc('font', family='sans-serif')
plt.rc('xtick.major', size=4, pad=7)
plt.rc('xtick', labelsize=10)
plt.rc('ytick.major', size=4, pad=7)
plt.rc('ytick', labelsize=10)
plt.rc('axes', titlesize=14)
plt.rc('axes', labelsize=12)
plt.rc('legend',**{'fontsize':12})
plt.rc('savefig', bbox='tight')
plt.rc('figure.subplot', wspace=0.3)
plt.rc('figure.subplot', hspace=0.3)
plt.rc('legend', fancybox=True, borderaxespad=0.)

import lsst.afw.table
import lsst.afw.image
import lsst.afw.math
import lsst.meas.algorithms
import lsst.meas.base
import lsst.meas.deblender
# import lsst.meas.extensions.shapeHSM
# from lsst.sims.GalSimInterface.wcsUtils import tanSipWcsFromDetector
# from lsst.sims.GalSimInterface import LSSTCameraWrapper
# from lsst.sims.utils import ObservationMetaData
schema = lsst.afw.table.SourceTable.makeMinimalSchema()
config1 = lsst.meas.algorithms.SourceDetectionConfig()
# Tweaks in the configuration to improve detection
#####
config1.tempLocalBackground.binSize=8
config1.minPixels=1
#config1.thresholdValue=12
#####
detect = lsst.meas.algorithms.SourceDetectionTask(schema=schema, config=config1)
deblend = lsst.meas.deblender.SourceDeblendTask(schema=schema)
config1 = lsst.meas.base.SingleFrameMeasurementConfig()
## HSM is not included in the stack by default. You have to download it and activate it.
#config1.plugins.names.add('ext_shapeHSM_HsmShapeBj')
#config1.plugins.names.add('ext_shapeHSM_HsmShapeLinear')
#config1.plugins.names.add('ext_shapeHSM_HsmShapeKsb')
# config1.plugins.names.add('ext_shapeHSM_HsmShapeRegauss')
#config1.plugins.names.add('ext_shapeHSM_HsmSourceMoments')
# config1.plugins.names.add('ext_shapeHSM_HsmPsfMoments')
measure = lsst.meas.base.SingleFrameMeasurementTask(schema=schema, config=config1)
# camera_wrapper = LSSTCameraWrapper()
# obs = ObservationMetaData(pointingRA=0, pointingDec=0,
#                          boundType='circle', boundLength=2.0,
#                          mjd=52000.0, rotSkyPos=0,
#                          bandpassName='i')

In [ ]:
data_dir = "/global/projecta/projectdirs/lsst/groups/WL/projects/wl-btf/two_gal_blend_data/" 

In [ ]:
def get_deblended_images(catalog, wldeb, masked_image):
    """Gets heavy footprint of all objects and 
    returns the deblended image of each child.
    """
    deb_images = []
    shape = wldeb.survey.image.array.shape
    for record in catalog:
        hfpt = get_hfpt(record, masked_image)
        #if hfpt is None:
        #    continue
        box=hfpt.getBBox()
        hfpt_image = lsst.afw.image.ImageF(box)
        hfpt.insert(hfpt_image)
        generator = galsim.random.BaseDeviate(seed=wldeb.noise_seed*100)
        noise = galsim.PoissonNoise(rng=generator, sky_level=wldeb.survey.mean_sky_level)
        before_0, after_0 = box.getBeginY(), shape[0] - box.getEndY()
        before_1, after_1 = box.getBeginX(), shape[1] - box.getEndX()
        im_temp = np.lib.pad(hfpt_image.array, ((before_0, after_0), (before_1, after_1)),
                             'constant')
        im_temp_galsim = galsim.Image(im_temp, scale=0.2)
        #im_temp_galsim.addNoise(noise)
        deb_images.append(im_temp_galsim.array)
    return deb_images
        
def get_hfpt(record, masked_image):
    """Gets Heavy footprint of a particular catalog object"""
    fpt = record.getFootprint()
    if fpt.isHeavy() is False:
        return lsst.afw.detection.makeHeavyFootprint(fpt, masked_image)
    return fpt

def process(input_path, output_path, run_DM, seed=123):
    wldeb = descwl.output.Reader(input_path).results # We read the image using descwl's package
    wldeb.add_noise(noise_seed=seed) # We add noise
    wldeb_image = wldeb.survey.image.array 
    image = lsst.afw.image.ImageF(wldeb_image) # We translate the image to be stack-readable
    # CHECK THIS!!
    if run_DM:
        print ("Running DM stack")
    else:
        return wldeb_image, []    
    variance_array = wldeb.survey.image.array # We generate a variance array
    variance = lsst.afw.image.ImageF(variance_array) # Generate the variance image
    masked_image = lsst.afw.image.MaskedImageF(image, None, variance) # Generate a masked image, i.e., an image+mask+variance image (with mask=None)
    psf_array = wldeb.survey.psf_image.array # We read the PSF image from the package
    psf_array = psf_array.astype(np.float64) 
    # psf_new = scipy.ndimage.zoom(psf_array,zoom=43/76.) # We have to rescale to match the stack's size
    psf_new = scipy.ndimage.shift(psf_array, shift=0.5)[16:61,16:61]
    im = lsst.afw.image.ImageD(psf_new) # Convert to stack's format
    fkernel = lsst.afw.math.FixedKernel(im) 
    psf = lsst.meas.algorithms.KernelPsf(fkernel) # Create the kernel in the stack's format
    exposure = lsst.afw.image.ExposureF(masked_image) # Passing the image to the stack
    exposure.setPsf(psf) # Assign the exposure the PSF that we created
    # wcs_in = tanSipWcsFromDetector('R:2,2 S:1,1',camera_wrapper,obs,2000) # We generate a WCS
    # exposure.setWcs(wcs_in) # And assign it to the exposure
    table = lsst.afw.table.SourceTable.make(schema)  # this is really just a factory for records, not a table
    detect_result = detect.run(table, exposure) # We run the stack (the detection task)
    catalog = detect_result.sources   # this is the actual catalog, but most of it's still empty
    deblend.run(exposure, catalog) # run the deblending task
    measure.run(catalog, exposure) # run the measuring task
    catalog = catalog.copy(deep=True)
    children = catalog[(catalog['deblend_nChild']==0)]
    deb_img = get_deblended_images(children,  wldeb, masked_image)
    catalog.writeFits(output_path) #write a copy of the catalog
    return image.array, deb_img # We return a catalog object

def get_a_b(e, hlr):
    """Returns semimajor/minor axis from ellipticity and HLR"""
    q = (1 - e) / (1. + e)
    b = hlr
    a = b / q
    return a, b
def get_g_from_e(e1 ,e2):
    g = np.ones([len(e1),2]) * -10
    for i in range(len(e1)):
        sh = galsim.Shear(e1 = e1[i], e2=e2[i])
        g[i] = [sh.g1, sh.g2]
    return g

def get_biased_g(g, bias):
    return g + bias

def get_ab_sigma(e, sigma):
    q = (1. - e) / (1. + e)
    b = sigma
    a = b / q
    return a, b

def get_p_angle(e):
    p = np.ones(len(e)) * -10
    for i in range(len(e)):
        p[i] =  math.atan2(e[i][1], e[i][0]) /2.
    return p * 180./np.pi

def print_vals(cat1, cat2):
    lab1=["purity", "base_SdssShape_flux"]
    lab2 = ["purity", "flux" ]
    disp_labels =['Purity', "Flux"]
    extra = ["snr_grpf", "snr_isof"]
    for i in range(len(lab1)):
        if cat1:
            val1 = f"{cat1[lab1[i]]:.2f}"
        else:
            val1=" - "
        if cat2:
            v = cat2[lab2[i]]
            val2 = f"{v:.2f}"
        else:
            val2=" - "
        display([val1, val2], disp_labels[i])
    if cat2:
        for i in range(len(extra)):
            v = cat2[extra[i]]
            display([' - ', f"{v:.2f}"], extra[i])
                 
def display(vals, name):
    print ("{0:<12} {1:<25} {2:>26}".format(name,vals[0], vals[1]))
        
def get_sigma(xx,yy,xy):
    """Return defined as |Q|^0.25, where Q is the second momemt"""
    return (xx * yy - xy**2)**0.25

def get_match(tru_cat, det_cat,
              im_shape, tolerance=5):
    z1 = np.zeros((len(tru_cat),2)) 
    z1[:,0] = np.array(tru_cat['dx']/0.2 + (im_shape[1]-1)/2.)
    z1[:,1] = np.array(tru_cat['dy']/0.2 + (im_shape[0]-1)/2.)
    z1_tree = spatial.KDTree(z1)
    z2 = np.zeros((len(det_cat),2)) 
    z2[:,0] = det_cat['base_GaussianCentroid_x']
    z2[:,1] = det_cat['base_GaussianCentroid_y']
    match = z1_tree.query(z2, distance_upper_bound=tolerance)
    return match

def plot_gals(images, label,vmax, vmin,
              xmin, xmax, ymin, ymax):
    plt.figure(figsize=[18,12])
    third_plot = True
    cat_labels =[": DM", ": WLDeb", "True - Obs" ]
    for i in range(2):
        plt.subplot(3, 3, i+1)
        if np.any(images[i]):
            plt.imshow(images[i][ymin:ymax, xmin:xmax],
                       vmin=vmin,vmax=vmax)
            plt.title(label + cat_labels[i])
            plt.colorbar(label='Counts')
            if i==0:
                mask = [images[i] == 0.]
        else:
            third_plot = False
    if third_plot:
        plt.subplot(3, 3, 3)
        diff = images[0] - images[1]
        diff[mask] = 0
        plt.imshow(diff[ymin:ymax, xmin:xmax],
                   vmin=-vmax*0.5,vmax=vmax*0.5,
                   cmap=plt.get_cmap('bwr'))
        plt.title("Obs - True")
        plt.colorbar(label='Counts')
    plt.show()

def plot_ellipses(im_blend, cat_wl, vmax, vmin,
                  x0, y0,xmin, xmax, ymin, ymax):
    g = get_g_from_e(cat_wl['e1'] ,cat_wl['e2'])
    g_new = get_biased_g(g , np.array([0,0]))
    g_new_mag = np.hypot(g_new.T[0], g_new.T[1])
    a_true, b_true =get_ab_sigma(g_new_mag, 0. + cat_wl['sigma_m'])
    p = get_p_angle(g_new)
    fig, ax = plt.subplots(1, 1)
    ax.imshow(im_blend[ymin:ymax, xmin:xmax],vmin=vmin,vmax=vmax)
    e1 = mpatches.Ellipse(((xmax - xmin-1)/2., (ymax - ymin-1)/2.),
                          a_true[0]/0.2, b_true[0]/0.2, p[0] ,
                          edgecolor='red', facecolor='none')
    ax.add_patch(e1)
    e2 = mpatches.Ellipse(((xmax - xmin-1)/2. + x0, (ymax - ymin-1)/2. + y0),
                          a_true[1]/0.2, b_true[1]/0.2, p[1] ,
                          edgecolor='red', facecolor='none')
    
    ax.add_patch(e2)

    g = get_g_from_e(cat_wl['e1'] ,cat_wl['e2'])
    g_new = get_biased_g(g , np.array([cat_wl['bias_g1_grp'] ,cat_wl['bias_g2_grp']]))
    g_new_mag = np.hypot(g_new.T[0], g_new.T[1])
    a_true, b_true =get_ab_sigma(g_new_mag, cat_wl['bias_s_grp']  +cat_wl['sigma_m'])
    p = get_p_angle(g_new)

    e3 = mpatches.Ellipse(((xmax - xmin-1)/2.+ cat_wl['bias_x_grp'][0], (ymax - ymin-1)/2.+ cat_wl['bias_y_grp'][0]),
                          a_true[0]/0.2, b_true[0]/0.2, p[0] ,
                          edgecolor='blue', facecolor='none')
    ax.add_patch(e3)
    e4 = mpatches.Ellipse(((xmax - xmin-1)/2. +cat_wl['bias_x_grp'][1] + x0, (ymax - ymin-1)/2. + y0 + cat_wl['bias_y_grp'][1]) ,
                          a_true[1]/0.2, b_true[1]/0.2, p[1] ,
                          edgecolor='blue', facecolor='none')
    ax.add_patch(e4)

    plt.show()

def run(flux_frac=0.75, bhlr_frac=1, dhlr_frac=1,
         x0=10, y0=5, p_angle=0, b_e=0, d_e = 0.,
         g1=0.01, g2=0.01, band='i', exp = 5520,
         run_DM=True):    
    im_h, im_w=225, 225 # image size
    x_size, y_size = 85,85 # Number of pixels to display 
    in_all = data_dir + 'gal_pair_catalog.fits'
    out_all = data_dir + 'gal_pair_wldeb.fits'
    in1 = data_dir + 'gal1_catalog.fits'
    in2 = data_dir + 'gal2_catalog.fits'
    out1 = data_dir + 'gal1_wldeb.fits'
    out2 = data_dir + 'gal2_wldeb.fits'
    dm_out_all = data_dir + 'gal_pair_dm.fits'
    dm_out1 = data_dir + 'gal1_dm.fits'
    dm_out2 = data_dir + 'gal2_dm.fits'
    
    if (np.abs(x0)>20 or (np.abs(y0) > 20)):
        print (" galaxy near stamp edge: Select |x0|<=20 and |y0|<=20 ")
        return
    if flux_frac <= 0:
        print (" flux_frac must be positive ")
        return

    % run make_catalog.py --flux_frac $flux_frac --bhlr_frac $bhlr_frac --dhlr_frac $dhlr_frac \
         --x0 $x0 --y0 $y0 --p_angle $p_angle --b_e $b_e --d_e $d_e --path $data_dir

    %run WeakLensingDeblending/simulate.py --catalog-name $in_all --output-name $out_all \
        --image-width $im_w --image-height $im_h --no-stamps --filter-band $band --exposure-time $exp \
        --calculate-bias --cosmic-shear-g1 $g1 --cosmic-shear-g2 $g2
    %run WeakLensingDeblending/simulate.py --catalog-name $in1 --output-name $out1 \
        --image-width $im_w --image-height $im_h --no-stamps --filter-band $band --exposure-time $exp \
        --calculate-bias --cosmic-shear-g1 $g1 --cosmic-shear-g2 $g2
    %run WeakLensingDeblending/simulate.py --catalog-name $in2 --output-name $out2 \
        --image-width $im_w --image-height $im_h --no-stamps --filter-band $band --exposure-time $exp \
        --calculate-bias --cosmic-shear-g1 $g1 --cosmic-shear-g2 $g2
    cat_wl = Table.read(out_all, format='fits', hdu=1)
    cat_wl1 = Table.read(out1, format='fits', hdu=1)
    cat_wl2 = Table.read(out2, format='fits', hdu=1)
    cat_wls = [cat_wl1, cat_wl1]
    
    im_blend, deb_imgs_blend = process(out_all, dm_out_all, run_DM)
    im1, deb_imgs1 = process(out1, dm_out1, run_DM=False)
    im2, deb_imgs2 = process(out2, dm_out2, run_DM=False)
    images = [im1, im2]
        
    if run_DM is True:
        cat = Table.read(dm_out_all,
                         format='fits', hdu=1)
        dm_cat_blend = cat[cat['deblend_nChild']==0]
        cat = Table.read(dm_out1,
                         format='fits', hdu=1)
        cat1 = cat[cat['deblend_nChild']==0]
        cat = Table.read(dm_out2,
                         format='fits', hdu=1)
        cat2 = cat[cat['deblend_nChild']==0]
        purity = 1/(1 + dm_cat_blend['base_Blendedness_old'])
        col = Column(purity, name="purity")
        dm_cat_blend.add_column(col)
        dm_catalogs = [cat1, cat2]
        match = get_match(cat_wl, dm_cat_blend,im_blend.shape)
        num = len(dm_cat_blend)
        print ("Number of children detected by DM", num)
    else:
        match = [np.array([np.nan, np.nan]),
                 np.array([0, 1])]
        num = 2
        print ("Not running DM")
        print ("Number of true galaxies", num)
        
    xmin, xmax = int((im_w - x_size)/2), int(im_w - (im_w - x_size)/2 )
    ymin, ymax = int((im_h - y_size)/2), int(im_h - (im_h - y_size)/2)
    vmax, vmin = im_blend.max() * 0.8, -10  
    # Plot blend
    plot_ellipses(im_blend, cat_wl, vmax, vmin,
                  x0,y0,xmin, xmax, ymin, ymax)
    # Plot individual galaxies
    labels = ['Central Galaxy', "Second Galaxy"]

    for i in range(num):
        if np.isinf(match[0][i]):
            plot_gals([deb_imgs_blend[i], None], 'Spurious Detection', vmax, vmin,
                      xmin, xmax, ymin, ymax)
            print ("Spurious Detection\n")
            print_vals(dm_cat_blend[i], None)
            print("\n\n")
        elif np.isnan(match[0][i]):
            plot_gals([None, images[match[1][i]]], labels[match[1][i]], vmax, vmin,
                      xmin, xmax, ymin, ymax)
            print_vals(None, cat_wl[match[1][i]])
        else:
            if num==1:
                plot_gals([im_blend, images[match[1][i]] ], labels[match[1][i]], vmax, vmin,
                          xmin, xmax, ymin, ymax)
                print_vals(dm_cat_blend[i], cat_wl[match[1][i]])
                undet_indx = int((match[1][i] + 1)%2)
                plot_gals([im_blend, images[undet_indx]], 'Undetected Object', vmax, vmin,
                          xmin, xmax, ymin, ymax)
                print_vals(None, cat_wl[undet_indx])
            else:
                plot_gals([deb_imgs_blend[i], images[match[1][i]] ], labels[match[1][i]], vmax, vmin,
                          xmin, xmax, ymin, ymax)
                print_vals(dm_cat_blend[i], cat_wl[match[1][i]])
    

    

In [ ]:
run()

In [ ]:
run(x0=12, y0=12, flux_frac=1, d_e=0, p_angle=180, run_DM=True)

In [ ]:
run(x0=20, y0=18, flux_frac=5, d_e=0.8, p_angle=180)

Q) 3 ways to get undetected object? Can you guess before running which galaxy will be undetected?

Ans:reduce distance, reduce flux, increse flux, icrease size

Q) get spurious detection

Ans:increase flux and ellipticity

## Analayze field of multiple 2 galaxy blends

In [ ]:
im_blend, deb_imgs_blend = process(data_dir + 'mock_gal_pairs_wldeb.fits',
                                   data_dir + 'mock_gal_pairs_dm.fits',
                                   run_DM=True)
dm_output_image = deb_imgs_blend[0]
for im in deb_imgs_blend[1:]:
    dm_output_image +=im

im_temp_galsim = galsim.Image(dm_output_image, scale=0.2)
galsim.fits.write(im_temp_galsim,
                  file_name=data_dir + 'mock_gal_pairs_dm_image.fits')

In [ ]:
def get_grp_size(cat, id_in):
    num = 0
    row = cat[cat['id'] == id_in]
    while (row['parent'] != 0):
        new_id = int(row['parent'])
        row = cat[cat['id'] == new_id]
        num += int(row['deblend_nChild'])
    return num
dm_all = Table.read(data_dir + 'mock_gal_pairs_dm.fits',
                    format= 'fits', hdu=1)
purity = 1/(1 + dm_all['base_Blendedness_old'])
col = Column(purity, name="purity")
dm_all.add_column(col)
grp_size = np.zeros(len(dm_all))
for i in range(len(dm_all)):
    grp_size[i] = get_grp_size(dm_all, dm_all["id"][i])
col = Column(grp_size, name="grp_size")
dm_all.add_column(col)

#iso = dm_all[dm_all['parent'] == 0]
#np.testing.assert_array_equal(iso['grp_size'], 0)

In [ ]:

dm_out = dm_all[dm_all["deblend_nChild"]==0]
dm_image = fitsio.read(data_dir + 'mock_gal_pairs_dm_image.fits',ext=0)
wl_out = Table.read(data_dir + 'mock_gal_pairs_wldeb.fits',
                    format= 'fits', hdu=1)
match = get_match(wl_out, dm_out,dm_image.shape)
select_dm, = np.where((~np.isinf(match[0])) &(~np.isnan(dm_out["base_SdssShape_flux"])))
select_wldeb = match[1][select_dm]
# match_dm = match[1][~np.isinf(match[0])]
print (f"Number of detected children is {len(dm_out)} out of {len(wl_out)} simulated galaxies")
print (f"Number of true matched detections is {len(select_dm)}")



In [ ]:
plt.figure(figsize=[14,10])
plt.subplot(3,3,1)
plt.scatter(wl_out['purity'][select_wldeb],
            dm_out['purity'][select_dm],
            c=dm_out["grp_size"][select_dm],
            vmin=0, vmax=10, alpha=0.4)
plt.plot([0.5,1], [0.5,1], 'k--')
plt.ylabel("Purity from DM")
plt.xlabel("True purity")
plt.colorbar()
plt.subplot(3,3,2)
plt.scatter(wl_out['flux'][select_wldeb],
            dm_out['base_SdssShape_flux'][select_dm],
           c=dm_out["grp_size"][select_dm],
            vmin=0, vmax=10, alpha=0.4)
plt.colorbar()
plt.ylabel("Flux from DM")
plt.xlabel("True Flux ")
lims = np.array([np.min(dm_out['base_SdssShape_flux'][select_dm]),
        np.max(dm_out['base_SdssShape_flux'][select_dm])])
plt.xlim(lims)
plt.ylim(lims)
plt.plot(lims, lims, 'k--')
plt.plot(lims, 2*lims, 'r--')

plt.subplot(3,3,3)
plt.scatter(wl_out['grp_size'][select_wldeb],
            dm_out['grp_size'][select_dm],
            alpha=0.4)
#plt.plot([0.5,1], [0.5,1], 'k--')
plt.ylabel("grp_size from DM")
plt.xlabel("True grp_size")
